In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
 


In [72]:
discount_rate = 0.0541
monthly_solar_radiation = [5.9*31, 6.4*28, 7*31, 7*30, 6.8*31, 6.9*30, 7.1*31, 7.2*31, 6.7*30, 6.1*31, 5.6*30, 5.3*31]
degradation_rate = 0.00469
electricity_tariff = 0.5444
feed_in_tariff = electricity_tariff*0.50
kw10 = 9.787
ic10 = 19667
kw100 = 99.920
ic100 = 180833


In [37]:
eg = list(i * 10 for i in monthly_solar_radiation)
eg.reverse()
print((eg))

[1642.9999999999998, 1680.0, 1891.0, 2010.0, 2232.0, 2201.0, 2070.0, 2108.0, 2100, 2170, 1792.0000000000002, 1829.0]


In [215]:
def NPV_formula(kw, ic, d, msr, dr, et, fit, eu):
    NPV =[]
    for l in eu:
        npv = -ic
        for i in range(25):
            yearly_value = 0
            monthly_electricity_generated = list(j * 10 * (1-dr*i) for j in msr) 
            for k in monthly_electricity_generated:
                monthly_value = -15*kw
                if k > l:
                    monthly_value += l * et
                    monthly_value += (k-l)*fit
                else:
                    monthly_value += k*et
                yearly_value += monthly_value
            npv+=(yearly_value/(1+d)**(i+1))
        NPV.append(round(npv))
    return NPV        

In [217]:
def SPP_formula(kw, ic, d, msr, dr, et, fit, eu):
    SPP = []
    for l in eu:
        NPV = -ic
        for i in range(25):
            yearly_value = 0
            monthly_electricity_generated = list(j * 10 * (1-dr*i) for j in msr) 
            for k in monthly_electricity_generated:
                monthly_value = -15*kw
                if k > l:
                    monthly_value += l * et
                    monthly_value += (k-l)*fit
                else:
                    monthly_value += k*et
                yearly_value += monthly_value
            if kw == 99.920:
                print(yearly_value)
            NPV+=(yearly_value/(1+d)**(i+1))
            if NPV > 0:
                spp = i+1
                break
        if NPV > 0:
            monthly_electricity_generated.reverse()
            month_counter = 1
            for k in monthly_electricity_generated:
                month_counter -= 0.083
                monthly_value = -15*kw
                if k > l:
                    monthly_value += l * et
                    monthly_value += (k-l)*fit
                else:
                    monthly_value += k*et
                NPV -= monthly_value/(1+d)**(i+1)
                if NPV < 0:
                    spp += round(month_counter,2)
                    monthly_electricity_generated.reverse()
                    break
            SPP.append(spp)
        else:
            SPP.append(0)
    return SPP

In [114]:
def IRR_formula(kw, ic, msr, dr, et, fit, eu):
    IRR = []
    for i in eu:
        lower_bound = 0.0
        upper_bound = 1.0 
        tolerance = 1e-4
        max_iterations = 1000
        iteration = 0

        while iteration < max_iterations:
            mid_rate = (lower_bound + upper_bound) / 2
            npv = NPV_formula(kw, ic, mid_rate, msr, dr, et, fit, i)

            if abs(npv) < tolerance: 
                IRR.append(round(mid_rate*100, 3))
                break
            elif npv > 0:
                lower_bound = mid_rate  
            else:
                upper_bound = mid_rate 
            iteration += 1 
    return IRR

In [193]:
def LCOE_formula(kw, ic, d, msr, dr, et, fit, eu):
    NPV_costs = ic
    NPV_energy = 0
    for i in range(25):
        NPV_costs += 12*15*kw/(1+d)**(i+1)
        NPV_energy += (sum(msr) * kw * (1 - dr*i)) / (1+d)**(i+1)
    return NPV_costs/NPV_energy
    

In [61]:
print(f"{round(IRR_formula(10, 19500, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, 1000),4)*100}%")

40.25%


In [64]:
print(f"{round(SPP_formula(10, 19500, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, 1000), 2)} years")


3.67 years


In [66]:
print(f"{round(LCOE_formula(10, 19500, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, 1000), 2)} AWG/kWh")

0.14 AWG/kWh


In [68]:
print(f"{round(NPV_formula(10, 19500, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, 1000), 2)} AWG")

83913.4 AWG


In [116]:
def IRR_graph(kw, ic, x_range):
    x = np.array(x_range)
    y = (IRR_formula(kw, ic, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x))
    df = pd.DataFrame({'x': x, 'y': y})
    
    fig = px.line(df, x='x', y='y', title=f'IRR of a {round(kw)}kWp system depending on electricity usage', labels={'x': 'Monthly Electricity Usage (kWh)', 'y': 'IRR in %'})
    
    fig.show()

In [205]:
def SPP_graph(x_range):
    x = np.array(x_range)
    y10 = (SPP_formula(kw10, ic10, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x))
    df10 = pd.DataFrame({'x': x, 'y': y10})

    y100 = (SPP_formula(kw100, ic100, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x))
    df100 = pd.DataFrame({'x': x, 'y': y100})

    fig = px.line(df10, x='x', y='y', title=f'SPP of a 10 and 100 kWp system depending on electricity usage', labels={'x': 'Monthly Electricity Usage (kWh)', 'y': 'SPP (years)'})

    fig.add_scatter(x=df10['x'], y=df10['y'], mode='lines', name='10 kWp System', line=dict(dash='solid'))

    fig.add_scatter(x=df100['x'], y=df100['y'], mode='lines', name='100 kWp System')

    fig.show()

In [198]:
def LCOE_graph(x_range):
    x = np.array(x_range)
    y10 = (LCOE_formula(kw10, ic10, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x))
    df10 = pd.DataFrame({'x': x, 'y': y10})

    y100 = (LCOE_formula(kw100, ic100, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x))
    df100 = pd.DataFrame({'x': x, 'y': y100})

    fig = px.line(df10, x='x', y='y', title=f'LCOE of a 10 and 100 kWp system depending on electricity usage', labels={'x': 'Monthly Electricity Usage (kWh)', 'y': 'LCOE (kWh/AWG)'})

    fig.add_scatter(x=df10['x'], y=df10['y'], mode='lines', name='10 kWp System', line=dict(dash='solid'))


    fig.add_scatter(x=df100['x'], y=df100['y'], mode='lines', name='100 kWp System')

    fig.update_yaxes(range=[0, 0.2])

    fig.show()

In [179]:
def NPV_graph(kw, ic, x_range):
    x = np.array(x_range)
    y = (NPV_formula(kw, ic, discount_rate, monthly_solar_radiation, degradation_rate, electricity_tariff, feed_in_tariff, x))
    df = pd.DataFrame({'x': x, 'y': y})
    
    fig = px.line(df, x='x', y='y', title=f'NPV of a {round(kw)}kWp system depending on electricity usage', labels={'x': 'Monthly Electricity Usage (kWh)', 'y': 'NPV (AWG)'})

    fig.show()

In [ ]:
IRR_graph(kw10, ic10, x_range=np.linspace(1000, 5000, 1000))

In [218]:
SPP_graph(x_range=np.linspace(1000, 100000, 1000))

-8260.982800000002
-8291.271838667999
-8321.560877335998
-8351.849916004001
-8382.138954672
-8412.427993340001
-8442.717032008
-8473.006070676001
-8503.295109343999
-8533.584148012
-8563.873186679999
-8594.162225348
-8624.451264016
-8654.740302684
-8685.029341352
-8715.318380019999
-8745.607418687998
-8775.896457355999
-8806.185496024002
-8836.474534692
-8866.76357336
-8897.052612028
-8927.341650695998
-8957.630689364
-8987.919728032002
-7937.285502702703
-7967.574541370703
-7997.863580038703
-8028.152618706704
-8058.441657374703
-8088.730696042703
-8119.019734710705
-8149.308773378703
-8179.5978120467025
-8209.886850714704
-8240.175889382701
-8270.464928050704
-8300.753966718703
-8331.043005386702
-8361.332044054703
-8391.621082722704
-8421.910121390702
-8452.199160058703
-8482.488198726702
-8512.777237394703
-8543.066276062704
-8573.355314730705
-8603.644353398704
-8633.933392066703
-8664.222430734702
-7613.588205405405
-7643.877244073405
-7674.166282741404
-7704.455321409404
-7734.7

In [ ]:
LCOE_graph(x_range=np.linspace(1000, 100000, 1000))

In [ ]:
NPV_graph(kw10, ic10, x_range=np.linspace(1000, 3000, 1000))
          